#### LSTM For EEG Data
- First, import everything we're gonna be using

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [77]:
class EEGLSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim = 4):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        self.hidden = self.init_hidden(self.hidden_dim)
        
        # hidden layer init
    def init_hidden(self, hidden_dim):
        """Initialize the hidden state in self.hidden
        Dimensions are num_layers * minibatch_size * hidden_dim
        """
        return (autograd.Variable(torch.zeros(1, 1, hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, hidden_dim)))
    
    def forward(self, input):
        # input is 1000 dimensional for now
        input = autograd.Variable(torch.FloatTensor(input)).contiguous()
        input = input.view(1, 1, -1) # present the sequence one at a time for now
        lstm_out, self.hidden = self.lstm(input, self.hidden)
        scores = self.linear(lstm_out.view(1,-1))
        scores = F.log_softmax(scores)
        return scores
        
        
    

In [78]:
model = EEGLSTM(22, 20)
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [66]:
# load the data
from load_data import EEGDataLoader
data_loader = EEGDataLoader()
X_train, y_train, X_test, y_test = data_loader.load_all_data()

In [79]:
sample = X_train[0][0].T
label = y_train[0][0]
model.zero_grad()
model.hidden = model.init_hidden(20)
print(label)
sample[0].shape
scores = model(sample[0])
print(scores.shape)
print(scores)
label = autograd.Variable(torch.LongTensor([int(label % 769)]))
loss = loss_function(scores, label)
print(loss.data[0])

loss.backward(retain_graph = True)
optimizer.step()
scores = model(sample[1])
loss = loss_function(scores, label)
print(loss.data[0])

model.zero_grad()
model.hidden = model.init_hidden(20)
loss.backward(retain_graph = True)
optimizer.step()
scores = model(sample[2])
loss = loss_function(scores, label)
print(loss.data[0])

771
torch.Size([1, 4])
Variable containing:
-1.2285 -1.3167 -1.6462 -1.4004
[torch.FloatTensor of size 1x4]

1.6461824178695679
1.8527568578720093
1.6217496395111084


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [80]:
for epoch in range(500):
    for i in range(X_train.shape[1]):
        # get the data
        model.zero_grad()
        model.hidden = model.init_hidden(20)
        X_sample, label = X_train[0][i].T[0], y_train[0][i]
        # clear out grads and re-init hidden state
        scores = model(X_sample)
        # compute the loss, grads, and update the parameters
        target = autograd.Variable(torch.LongTensor([int(label % 769)]))
        loss = loss_function(scores, target)
        if i % 10 == 0:
            print(loss.data[0])
        loss.backward()
        optimizer.step()
    
        
        
        
        
    

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


1.6361920833587646
1.4216657876968384
1.3514044284820557
1.5305604934692383
1.8480525016784668
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
n

KeyboardInterrupt: 